# Train a YOLO model for pose estimation on a custom keypoints dataset

This notebook uses the table loaded in [create-custom-keypoints-table.ipynb](../1-create-tables/create-custom-keypoints-table.ipynb), and trains a YOLO model for pose estimation on it.

## Project setup

In [2]:
PROJECT_NAME = "3LC Tutorials"
DATASET_NAME = "AnimalPose"
TABLE_NAME = "initial"

## Imports

In [3]:
from tlc_ultralytics import YOLO, Settings
from tlc_tools.split import split_table
import tlc

## Load and split table


In [4]:
initial_table = tlc.Table.from_names(TABLE_NAME, DATASET_NAME, PROJECT_NAME)
tables = split_table(initial_table, splits={"train": 0.8, "val": 0.2})

3lc: Loaded project alias configuration from C:/Users/gudbrand/AppData/Local/3LC/3LC/projects/3LC Tutorials/default_aliases.3lc.yaml


## Train model

In [5]:
model = YOLO("yolo11n-pose.pt")

settings = Settings(
    project_name=PROJECT_NAME,
    run_name="train-yolon-animalpose",
    run_description="Training a YOLO model for pose estimation on the AnimalPose dataset",
    collect_loss=True,
    image_embeddings_dim=2,
)

model.train(
    tables=tables,
    epochs=10,
    workers=0,
)

Using 3LC Trainer 
Ultralytics 8.3.184  Python-3.12.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/ultralytics/ultralytics/cfg/datasets/coco-pose.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train28, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_m

train: Preparing data from C:/Users/gudbrand/AppData/Local/3LC/3LC/projects/3LC Tutorials/datasets/AnimalPose/tables/train_0000 3687 images, 0 corrupt: 100%|██████████| 3687/3687 [00:01<00:00, 2371.71it/s]


train: Fast image access  (ping: 0.10.1 ms, read: 146.7179.3 MB/s, size: 65.1 KB)
WARNING No 'flip_idx' array defined in data.yaml, disabling 'fliplr' and 'flipud' augmentations.


val: Preparing data from C:/Users/gudbrand/AppData/Local/3LC/3LC/projects/3LC Tutorials/datasets/AnimalPose/tables/val_0000 921 images, 0 corrupt: 100%|██████████| 921/921 [00:00<00:00, 2846.00it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 528.1424.3 MB/s, size: 81.8 KB)
Plotting labels to c:\Project\3lc-ultralytics\runs\pose\train28\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 87 weight(decay=0.0), 97 weight(decay=0.0005), 96 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to c:\Project\3lc-ultralytics\runs\pose\train28
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...

    AnimalPose   Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 116/116 [00:54<00:00,  2.14it/s]


                   all       3687       4928      0.927      0.886      0.953      0.774      0.711      0.608      0.575       0.21
                   dog       3687       4928      0.927      0.886      0.953      0.774      0.711      0.608      0.575       0.21
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to c:\Project\3lc-ultralytics\runs\pose\train28
Optimizer stripped from c:\Project\3lc-ultralytics\runs\pose\train28\weights\last.pt, 6.2MB
Optimizer stripped from c:\Project\3lc-ultralytics\runs\pose\train28\weights\best.pt, 6.2MB

Validating c:\Project\3lc-ultralytics\runs\pose\train28\weights\best.pt...
Ultralytics 8.3.184  Python-3.12.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080 Laptop GPU, 12282MiB)
YOLO11n-pose summary (fused): 109 layers, 2,933,587 parameters, 0 gradients, 7.7 GFLOPs


    AnimalPose   Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:14<00:00,  2.00it/s]


                   all        921       1189      0.918      0.864       0.94      0.723      0.665      0.571      0.543      0.202
                   dog        921       1189      0.918      0.864       0.94      0.723      0.665      0.571      0.543      0.202
Speed: 0.3ms preprocess, 2.4ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to c:\Project\3lc-ultralytics\runs\pose\train28


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000213B224D8E0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.0340